
# Unfolded WMMSE versus WMMSE
The WMMSE algorithm is implemented as described in the paper "An Iteratively Weighted MMSE Approach to Distributed Sum-Utility Maximization for 
a MIMO Interfering Broadcast Channel", by Q. Shi et al. 

Note:
*   Functions and variables with "_nn" (i.e. neural network) in the name refer to the unfolded WMMSE algorithm
*   In the code we use the term precoder to indicate the beamformer 


### Import libraries and set variables

In [0]:
# Import libraries
import tensorflow as tf # tensorflow_version 1.x is needed
import numpy as np
import copy
from copy import deepcopy
import time
import matplotlib.pyplot as plt


# Set variables 
nr_of_users = 4
nr_of_BS_antennas = 4
scheduled_users = [0,1,2,3] # array of scheduled users. Note that we schedule all the users.
epsilon = 0.0001 # used to end the iterations of the WMMSE algorithm in Shi et al. when the number of iterations is not fixed (note that the stopping criterion has precendence over the fixed number of iterations)
power_tolerance = 0.0001 # used to end the bisection search in the WMMSE algorithm in Shi et al.
total_power = 100 # power constraint in the weighted sum rate maximization problem - eq. (4) in our paper
noise_power = 1
nr_of_batches_training = 10000 # used for training
nr_of_batches_test = 1000 # used for testing
nr_of_samples_per_batch = 100 
nr_of_iterations = 1 # for WMMSE algorithm in Shi et al. 
nr_of_iterations_nn = 1 # for the unfolded WMMSE in our paper

# User weights in the weighted sum rate (denoted by alpha in eq. (4) in our paper)
user_weights = np.reshape(np.ones(nr_of_users*nr_of_samples_per_batch),(nr_of_samples_per_batch,nr_of_users,1))
user_weights_for_regular_WMMSE = np.ones(nr_of_users)


### Define utility functions


In [0]:
# Compute power for bisection search in the optimization of the transmitter precoder 
# - eq. (18) in the paper by Shi et al.
def compute_P(Phi_diag_elements, Sigma_diag_elements, mu):
  nr_of_BS_antennas = Phi_diag_elements.size
  mu_array = mu*np.ones(Phi_diag_elements.size)
  result = np.divide(Phi_diag_elements,(Sigma_diag_elements + mu_array)**2)
  result = np.sum(result)
  return result


def compute_norm_of_complex_array(x):
  result = np.sqrt(np.sum((np.absolute(x))**2))
  return result


def compute_sinr(channel, precoder, noise_power, user_id, selected_users):
    nr_of_users = np.size(channel,0)
    numerator = (np.absolute(np.matmul(np.conj(channel[user_id,:]),precoder[user_id,:])))**2

    inter_user_interference = 0
    for user_index in range(nr_of_users):
      if user_index != user_id and user_index in selected_users:
        inter_user_interference = inter_user_interference + (np.absolute(np.matmul(np.conj(channel[user_id,:]),precoder[user_index,:])))**2
    denominator = noise_power + inter_user_interference

    result = numerator/denominator
    return result


def compute_user_weights(nr_of_users, selected_users):
  result = np.ones(nr_of_users)
  for user_index in range(nr_of_users):
    if not (user_index in selected_users):
      result[user_index] = 0
  return result


def compute_weighted_sum_rate(user_weights, channel, precoder, noise_power, selected_users):
   result = 0
   nr_of_users = np.size(channel,0)
   
   for user_index in range(nr_of_users):
     if user_index in selected_users:
       user_sinr = compute_sinr(channel, precoder, noise_power, user_index, selected_users)
       result = result + user_weights[user_index]*np.log2(1 + user_sinr)
    
   return result


def compute_sinr_nn(channel, precoder, noise_power, user_id, nr_of_users):

    numerator = tf.reduce_sum((tf.matmul(tf.transpose(channel[user_id]),precoder[user_id]))**2)
    inter_user_interference = 0
    for user_index in range(nr_of_users):
      if user_index != user_id:
        inter_user_interference = inter_user_interference +  tf.reduce_sum((tf.matmul(tf.transpose(channel[user_id]),precoder[user_index]))**2)
    denominator = noise_power + inter_user_interference

    result = numerator/denominator
    return result


def compute_WSR_nn(user_weights, channel, precoder, noise_power, nr_of_users):

   result = 0
   for batch_index in range(nr_of_samples_per_batch):
    for user_index in range(nr_of_users):
        user_sinr = compute_sinr_nn(channel[batch_index], precoder[batch_index], noise_power, user_index,nr_of_users)
        result = result + user_weights[batch_index][user_index]*(tf.log(1 + user_sinr)/tf.log(tf.cast(2.0,tf.float64)))

   return result


# Computes a channel realization and returns it in two formats, one for the WMMSE and one for the unfolded WMMSE.
# It also returns the initialization value of the transmitter precoder, which is used as input in the computation graph of the unfolded WMMSE.
def compute_channel(nr_of_BS_antennas, nr_of_users, total_power):
  channel_nn = []
  initial_transmitter_precoder = []
  channel_WMMSE = np.zeros((nr_of_users, nr_of_BS_antennas)) + 1j*np.zeros((nr_of_users, nr_of_BS_antennas))

  for i in range(nr_of_users):
      result_real = np.sqrt(0.5)*np.random.normal(size = (nr_of_BS_antennas,1))
      result_imag  =  np.sqrt(0.5)*np.random.normal(size = (nr_of_BS_antennas,1))
      channel_WMMSE[i,:] = np.reshape(result_real,(1,nr_of_BS_antennas)) + 1j*np.reshape(result_imag, (1,nr_of_BS_antennas))
      result_col_1 = np.vstack((result_real,result_imag))
      result_col_2 = np.vstack((-result_imag,result_real))
      result =  np.hstack((result_col_1, result_col_2))
      initial_transmitter_precoder.append(result_col_1)
      channel_nn.append(result)

  initial_transmitter_precoder_array = np.array(initial_transmitter_precoder)
  initial_transmitter_precoder_array = np.sqrt(total_power)*initial_transmitter_precoder_array/np.linalg.norm(initial_transmitter_precoder_array)
  initial_transmitter_precoder = []

  for i in range(nr_of_users):
    initial_transmitter_precoder.append(initial_transmitter_precoder_array[i])

  return channel_nn, initial_transmitter_precoder, channel_WMMSE


def layer_pg( init, name, mse_weights, user_weights, receiver_precoder,channel, initial_transmitter_precoder, total_power):

    with tf.variable_scope(name): 

      step_size =  tf.Variable(tf.constant(init, dtype=tf.float64), name="step_size", dtype=tf.float64)

      # First iteration
      a1_exp = tf.tile(tf.expand_dims(mse_weights[:,0,:],-1),[1,2*nr_of_BS_antennas,2*nr_of_BS_antennas])
      a2_exp = tf.tile(tf.expand_dims(user_weights[:,0,:],-1),[1,2*nr_of_BS_antennas,2*nr_of_BS_antennas])
      a3_exp = tf.tile(tf.expand_dims((tf.reduce_sum(receiver_precoder[:,0,:,:],axis =-2))**2,-1),[1,2*nr_of_BS_antennas,2*nr_of_BS_antennas])    
      temp = a1_exp*a2_exp*a3_exp*tf.matmul(channel[:,0,:,:],tf.transpose(channel[:,0,:,:],perm = [0,2,1]))
      
      # Next iterations
      for i in range(1, nr_of_users):
        a1_exp = tf.tile(tf.expand_dims(mse_weights[:,i,:],-1),[1,2*nr_of_BS_antennas,2*nr_of_BS_antennas])
        a2_exp = tf.tile(tf.expand_dims(user_weights[:,i,:],-1),[1,2*nr_of_BS_antennas,2*nr_of_BS_antennas])
        a3_exp = tf.tile(tf.expand_dims((tf.reduce_sum(receiver_precoder[:,i,:,:],axis =-2))**2,-1),[1,2*nr_of_BS_antennas,2*nr_of_BS_antennas])
        temp = temp + a1_exp*a2_exp*a3_exp*tf.matmul(channel[:,i,:,:],tf.transpose(channel[:,i,:,:],perm = [0,2,1]))

      sum_gradient = temp # it corresponds to eq. (6) in our paper

      gradient = []

      # Gradient computation
      for i in range(nr_of_users):
        a1_exp = tf.tile(tf.expand_dims(mse_weights[:,i,:],-1),[1,2*nr_of_BS_antennas,1])
        a2_exp = tf.tile(tf.expand_dims(user_weights[:,i,:],-1),[1,2*nr_of_BS_antennas,1])
        gradient.append(step_size*(-2.0*a1_exp*a2_exp*tf.matmul(channel[:,i,:,:],receiver_precoder[:,i,:,:])+ 2*tf.matmul(sum_gradient,initial_transmitter_precoder[:,i,:,:]))) 
        
      tf.stack(gradient)
      gradient = tf.transpose( tf.stack(gradient), perm=[1,0,2,3])
      output_temp = initial_transmitter_precoder - gradient # it corresponds to eq. (12a) in our paper

      output = []
      for i in range(nr_of_samples_per_batch):
        output.append(tf.cond((tf.linalg.norm(output_temp[i]))**2 < total_power, lambda: output_temp[i] , lambda: tf.sqrt(tf.cast(total_power, tf.float64))*output_temp[i]/tf.linalg.norm(output_temp[i]))) # it corresponds to eq. (13) in our paper

      return tf.stack(output), step_size



### Function to run the WMMSE algorithm described in the paper by Shi et al.

In [0]:
def run_WMMSE(epsilon, channel, selected_users, total_power, noise_power, user_weights, max_nr_of_iterations, log = False):

  nr_of_users = np.size(channel,0)
  nr_of_BS_antennas = np.size(channel,1)
  WSR=[] # to check if the WSR (our cost function) increases at each iteration of the WMMSE
  break_condition = epsilon + 1 # break condition to stop the WMMSE iterations and exit the while
  receiver_precoder = np.zeros(nr_of_users) + 1j*np.zeros(nr_of_users) # receiver_precoder is "u" in the paper of Shi et al. (it's a an array of complex scalars)
  mse_weights = np.ones(nr_of_users) # mse_weights is "w" in the paper of Shi et al. (it's a an array of real scalars)
  transmitter_precoder = np.zeros((nr_of_users, nr_of_BS_antennas)) + 1j*np.zeros((nr_of_users, nr_of_BS_antennas))# transmitter_precoder is "v" in the paper of Shi et al. (it's a complex matrix)
  
  new_receiver_precoder = np.zeros(nr_of_users) + 1j*np.zeros(nr_of_users) # for the first iteration 
  new_mse_weights = np.zeros(nr_of_users) # for the first iteration
  new_transmitter_precoder = np.zeros((nr_of_users, nr_of_BS_antennas)) + 1j*np.zeros((nr_of_users, nr_of_BS_antennas)) # for the first iteration

  
  # Initialization of transmitter precoder
  for user_index in range(nr_of_users):
    if user_index in selected_users:
      transmitter_precoder[user_index,:] = channel[user_index,:]
  transmitter_precoder = transmitter_precoder/np.linalg.norm(transmitter_precoder)*np.sqrt(total_power)
  
  # Store the WSR obtained with the initialized trasmitter precoder    
  WSR.append(compute_weighted_sum_rate(user_weights, channel, transmitter_precoder, noise_power, selected_users))

  # Compute the initial power of the transmitter precoder
  initial_power = 0
  for user_index in range(nr_of_users):
    if user_index in selected_users:
      initial_power = initial_power + (compute_norm_of_complex_array(transmitter_precoder[user_index,:]))**2 
  if log == True:
    print("Power of the initialized transmitter precoder:", initial_power)

  nr_of_iteration_counter = 0 # to keep track of the number of iteration of the WMMSE

  while break_condition >= epsilon and nr_of_iteration_counter<=max_nr_of_iterations:
    
    nr_of_iteration_counter = nr_of_iteration_counter + 1
    if log == True:
      print("WMMSE ITERATION: ", nr_of_iteration_counter)

    # Optimize receiver precoder - eq. (5) in the paper of Shi et al.
    for user_index_1 in range(nr_of_users):
      if user_index_1 in selected_users:
        user_interference = 0.0
        for user_index_2 in range(nr_of_users):
          if user_index_2 in selected_users:
            user_interference = user_interference + (np.absolute(np.matmul(np.conj(channel[user_index_1,:]),transmitter_precoder[user_index_2,:])))**2

        new_receiver_precoder[user_index_1] = np.matmul(np.conj(channel[user_index_1,:]),transmitter_precoder[user_index_1,:]) / (noise_power + user_interference)

    # Optimize mse_weights - eq. (13) in the paper of Shi et al.
    for user_index_1 in range(nr_of_users):
      if user_index_1 in selected_users:

        user_interference = 0 # it includes the channel of all selected users
        inter_user_interference = 0 # it includes the channel of all selected users apart from the current one
        
        for user_index_2 in range(nr_of_users):
          if user_index_2 in selected_users:
            user_interference = user_interference + (np.absolute(np.matmul(np.conj(channel[user_index_1,:]),transmitter_precoder[user_index_2,:])))**2
        for user_index_2 in range(nr_of_users):
          if user_index_2 != user_index_1 and user_index_2 in selected_users:
            inter_user_interference = inter_user_interference + (np.absolute(np.matmul(np.conj(channel[user_index_1,:]),transmitter_precoder[user_index_2,:])))**2
        
        new_mse_weights[user_index_1] = (noise_power + user_interference)/(noise_power + inter_user_interference)

    A = np.zeros((nr_of_BS_antennas,nr_of_BS_antennas))+1j*np.zeros((nr_of_BS_antennas,nr_of_BS_antennas))
    for user_index in range(nr_of_users):
      if user_index in selected_users:
        # hh should be an hermitian matrix of size (nr_of_BS_antennas X nr_of_BS_antennas)
        hh = np.matmul(np.reshape(channel[user_index,:],(nr_of_BS_antennas,1)),np.conj(np.transpose(np.reshape(channel[user_index,:],(nr_of_BS_antennas,1)))))
        A = A + (new_mse_weights[user_index]*user_weights[user_index]*(np.absolute(new_receiver_precoder[user_index]))**2)*hh

    Sigma_diag_elements_true, U = np.linalg.eigh(A)
    Sigma_diag_elements = copy.deepcopy(np.real(Sigma_diag_elements_true))
    Lambda = np.zeros((nr_of_BS_antennas,nr_of_BS_antennas)) + 1j*np.zeros((nr_of_BS_antennas,nr_of_BS_antennas))
    
    for user_index in range(nr_of_users):
      if user_index in selected_users:     
        hh = np.matmul(np.reshape(channel[user_index,:],(nr_of_BS_antennas,1)),np.conj(np.transpose(np.reshape(channel[user_index,:],(nr_of_BS_antennas,1)))))
        Lambda = Lambda + ((user_weights[user_index])**2)*((new_mse_weights[user_index])**2)*((np.absolute(new_receiver_precoder[user_index]))**2)*hh

    Phi = np.matmul(np.matmul(np.conj(np.transpose(U)),Lambda),U)
    Phi_diag_elements_true = np.diag(Phi)
    Phi_diag_elements = copy.deepcopy(Phi_diag_elements_true)
    Phi_diag_elements = np.real(Phi_diag_elements)

    for i in range(len(Phi_diag_elements)):
      if Phi_diag_elements[i]<np.finfo(float).eps:
        Phi_diag_elements[i] = np.finfo(float).eps
      if (Sigma_diag_elements[i])<np.finfo(float).eps:
        Sigma_diag_elements[i] = 0

    # Check if mu = 0 is a solution (eq.s (15) and (16) of in the paper of Shi et al.)
    power = 0 # the power of transmitter precoder (i.e. sum of the squared norm)
    for user_index in range(nr_of_users):
      if user_index in selected_users:
        if np.linalg.det(A) != 0:
          w = np.matmul(np.linalg.inv(A),np.reshape(channel[user_index,:],(nr_of_BS_antennas,1)))*user_weights[user_index]*new_mse_weights[user_index]*(new_receiver_precoder[user_index])
          power = power + (compute_norm_of_complex_array(w))**2

    # If mu = 0 is a solution, then mu_star = 0
    if np.linalg.det(A) != 0 and power <= total_power:
      mu_star = 0
    # If mu = 0 is not a solution then we search for the "optimal" mu by bisection
    else:
      power_distance = [] # list to store the distance from total_power in the bisection algorithm 
      mu_low = np.sqrt(1/total_power*np.sum(Phi_diag_elements))
      mu_high = 0
      low_point = compute_P(Phi_diag_elements, Sigma_diag_elements, mu_low)
      high_point = compute_P(Phi_diag_elements, Sigma_diag_elements, mu_high)

      obtained_power = total_power + 2*power_tolerance # initialization of the obtained power such that we enter the while 

      # Bisection search
      while np.absolute(total_power - obtained_power) > power_tolerance:
        mu_new = (mu_high + mu_low)/2
        obtained_power = compute_P(Phi_diag_elements, Sigma_diag_elements, mu_new) # eq. (18) in the paper of Shi et al.
        power_distance.append(np.absolute(total_power - obtained_power))
        if obtained_power > total_power:
          mu_high = mu_new
        if obtained_power < total_power:
          mu_low = mu_new
      mu_star = mu_new
      if log == True:
        print("first value:", power_distance[0])
        plt.title("Distance from the target value in bisection (it should decrease)")
        plt.plot(power_distance)
        plt.show()

    for user_index in range(nr_of_users):
      if user_index in selected_users:
        new_transmitter_precoder[user_index,:] = np.matmul(np.linalg.inv(A + mu_star*np.eye(nr_of_BS_antennas)),channel[user_index,:])*user_weights[user_index]*new_mse_weights[user_index]*(new_receiver_precoder[user_index]) 

    # To select only the weights of the selected users to check the break condition
    mse_weights_selected_users = []
    new_mse_weights_selected_users = []
    for user_index in range(nr_of_users): 
      if user_index in selected_users:
        mse_weights_selected_users.append(mse_weights[user_index])
        new_mse_weights_selected_users.append(new_mse_weights[user_index])

    mse_weights = deepcopy(new_mse_weights)
    transmitter_precoder = deepcopy(new_transmitter_precoder)
    receiver_precoder = deepcopy(new_receiver_precoder)

    WSR.append(compute_weighted_sum_rate(user_weights, channel, transmitter_precoder, noise_power, selected_users))
    break_condition = np.absolute(np.log2(np.prod(new_mse_weights_selected_users))-np.log2(np.prod(mse_weights_selected_users)))

  if log == True:
    plt.title("Change of the WSR at each iteration of the WMMSE (it should increase)")
    plt.plot(WSR,'bo')
    plt.show()

  return transmitter_precoder, receiver_precoder, mse_weights, WSR[-1]


### Tensorflow computation graph to run the unfolded WMMSE algorithm described in our paper

In [0]:
tf.reset_default_graph()

channel_input = tf.placeholder(tf.float64, shape = None, name = 'channel_input')
initial_tp = tf.placeholder(tf.float64, shape = None, name = 'initial_transmitter_precoder')
initial_transmitter_precoder = initial_tp

# Arrays that contain the initialization values of the step sizes.
# The number of step sizes depends on the selected nuber of PGD layers, the number of elements for each step size initializer depends on the selected number of unfolded iterations
step_size_init_1 = [1.0]
step_size_init_2 = [1.0]
step_size_init_3 = [1.0]
step_size_init_4 = [1.0]

# Used to collect the step sizes at each iteration
all_step_size1_temp = []
all_step_size2_temp = []
all_step_size3_temp = []
all_step_size4_temp = []

profit = [] # stores the WSR obtained at each iteration

for loop in range(0,nr_of_iterations_nn):

  user_interference2 = []

  for batch_index in range(nr_of_samples_per_batch): 
    user_interference_single = []
    for i in range(nr_of_users):
      temp = 0.0
      for j in range(nr_of_users):
        temp = temp + tf.reduce_sum((tf.matmul(tf.transpose(channel_input[batch_index, i,:,:]),initial_transmitter_precoder[batch_index,j,:,:]))**2)
      user_interference_single.append(temp + noise_power) 
    user_interference2.append(user_interference_single)
    
  tf.stack(user_interference2)

  user_interference_exp2 = tf.tile(tf.expand_dims(tf.tile(tf.expand_dims(user_interference2,-1),[1,1,2]),-1),[1,1,1,1])

  receiver_precoder_temp = (tf.matmul(tf.transpose(channel_input,perm = [0,1,3,2]),initial_transmitter_precoder))
  # Optimize the receiver precoder - eq. (5b) in our paper
  receiver_precoder = tf.divide(receiver_precoder_temp,user_interference_exp2)

  # Optimize the mmse weights - eq. (5a) in our paper
  self_interference = tf.reduce_sum((tf.matmul(tf.transpose(channel_input,perm = [0,1,3,2]),initial_transmitter_precoder))**2, axis = 2)

  inter_user_interference_total = []

  for batch_index in range(nr_of_samples_per_batch):
    inter_user_interference_temp = []
    for i in range(nr_of_users):
      temp = 0.0
      for j in range(nr_of_users):
        if j != i:
          temp = temp + tf.reduce_sum((tf.matmul(tf.transpose(channel_input[batch_index, i,:,:]),initial_transmitter_precoder[batch_index,j,:,:]))**2)
      inter_user_interference_temp.append(temp + noise_power) # $sum{|(h_i)*H,v_i}|**2 + noise_power$
    inter_user_interference = tf.reshape(tf.stack(inter_user_interference_temp),(nr_of_users,1)) # Nx1 $sum{|(h_i)*H,v_i}|**2 + noise_power$
    inter_user_interference_total.append(inter_user_interference)
      
  mse_weights = (tf.divide(self_interference,inter_user_interference_total)) + 1.0

  # Optimize the transmitter precoder through PGD (here denoted as "pg")
  transmitter_precoder1, step_size1 = layer_pg( step_size_init_1[loop],'pg1', mse_weights, user_weights, receiver_precoder,channel_input, initial_transmitter_precoder, total_power)
  transmitter_precoder2, step_size2 = layer_pg( step_size_init_2[loop],'pg2', mse_weights, user_weights, receiver_precoder,channel_input, transmitter_precoder1, total_power)
  transmitter_precoder3, step_size3 = layer_pg( step_size_init_3[loop],'pg3', mse_weights, user_weights, receiver_precoder,channel_input, transmitter_precoder2, total_power)
  transmitter_precoder, step_size4 = layer_pg( step_size_init_4[loop],'pg4', mse_weights, user_weights, receiver_precoder,channel_input, transmitter_precoder3, total_power)

  initial_transmitter_precoder = transmitter_precoder

  all_step_size1_temp.append(step_size1)
  all_step_size2_temp.append(step_size2)
  all_step_size3_temp.append(step_size3)
  all_step_size4_temp.append(step_size4)


  # The WSR achieved with the transmitter precoder obtained at the current iteration is appended
  profit.append(compute_WSR_nn(user_weights, channel_input, initial_transmitter_precoder, noise_power,nr_of_users))

all_step_size1 = tf.stack(all_step_size1_temp)
all_step_size2 = tf.stack(all_step_size2_temp)
all_step_size3 = tf.stack(all_step_size3_temp)
all_step_size4 = tf.stack(all_step_size4_temp)


final_precoder = initial_transmitter_precoder # this is the last transmitter precoder, i.e. the one that will be actually used for transmission

WSR = tf.reduce_sum(tf.stack(profit)) # this is the cost function to maximize, i.e. the WSR obtained if we use the transmitter precoder that we have at each round of the loop 
WSR_final = compute_WSR_nn(user_weights, channel_input, final_precoder, noise_power,nr_of_users)/nr_of_samples_per_batch # this is the WSR computed using the "final_precoder"

optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(-WSR)


### Running the unfolded WMMSE and the WMMSE
In this code we train the neural network and we compare its performance with the WMMSE, by testing both algorithms on the same dataset of channel realizations.

In [0]:
WSR_WMMSE =[]
WSR_nn = []
training_loss = []

with tf.Session() as sess:
    print("start of session")
    start_of_time = time.time()
    sess.run(tf.global_variables_initializer())
    
    for i in range(nr_of_batches_training):#500000 # for  10 000 we it takes 404 s
      batch_for_training = []
      initial_transmitter_precoder_batch = []

      # Building a batch for training
      for ii in range(nr_of_samples_per_batch):
        channel_realization_nn,init_transmitter_precoder,_ = compute_channel(nr_of_BS_antennas, nr_of_users,total_power)
        batch_for_training.append(channel_realization_nn)
        initial_transmitter_precoder_batch.append(init_transmitter_precoder)
      # Training
      sess.run(optimizer, feed_dict={channel_input:batch_for_training,initial_tp:initial_transmitter_precoder_batch })
      training_loss.append(-1*(sess.run(WSR, feed_dict={channel_input:batch_for_training,initial_tp:initial_transmitter_precoder_batch })))
    
    print("step size1", sess.run(all_step_size1))
    print("step size2", sess.run(all_step_size2))
    print("step size3", sess.run(all_step_size3))
    print("step size4", sess.run(all_step_size4))

    print("Training took:", time.time()-start_of_time)

    # For repeatability
    np.random.seed(1234)

    for i in range(nr_of_batches_test):    
      batch_for_testing = []
      initial_transmitter_precoder_batch = []
      WSR_WMMSE_batch = 0.0

      # Building a batch for testing
      for ii in range(nr_of_samples_per_batch):       
        channel_realization_nn,init_transmitter_precoder,channel_realization_regular = compute_channel(nr_of_BS_antennas, nr_of_users,total_power)    
        _,_,_,WSR_WMMSE_one_sample = run_WMMSE(epsilon, channel_realization_regular, scheduled_users, total_power, noise_power, user_weights_for_regular_WMMSE, nr_of_iterations-1, log = False)      
        WSR_WMMSE_batch =  WSR_WMMSE_batch + WSR_WMMSE_one_sample
        batch_for_testing.append(channel_realization_nn)
        initial_transmitter_precoder_batch.append(init_transmitter_precoder)
      #Testing
      WSR_nn.append(sess.run(WSR_final , feed_dict={channel_input:batch_for_testing,initial_tp:initial_transmitter_precoder_batch }))
      WSR_WMMSE.append(WSR_WMMSE_batch/nr_of_samples_per_batch)
    
print("Training and testing took:", time.time()-start_of_time)
print("The WSR acheived with the unfolded WMMSE algorithm is: ",np.mean(WSR_nn))
print("The WSR acheived with the WMMSE algorithm is: ",np.mean(WSR_WMMSE))

plt.figure()
plt.plot(training_loss)
plt.ylabel("Training loss")
plt.xlabel("Sample index")
